<a href="https://colab.research.google.com/github/michelnuss/Michel-Nussbacher/blob/main/Calculate_a_stock_ATR_graph_and_return.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import scipy.stats as sci
import numpy as np

def calculate_atr(ativo, atr_n):
    # Calcula os dados necessários para o TR:
    high_low = abs(ativo['High'] - ativo['Low'])
    high_close = abs(ativo['High'] - ativo['Close'].shift())
    low_close = abs(ativo['Low'] - ativo['Close'].shift())

    # Combina as componentes para calcular o True Range:
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    ativo[f'TR-{atr_n}'] = ranges.max(axis=1)

    # Cálculo do ATR
    ativo[f'ATR-{atr_n}'] = ativo[f'TR-{atr_n}'].rolling(window=atr_n).mean()

    # Cálculo dos ATR Seguintes usando EWM
    ativo[f'ATR-{atr_n}'] = ativo[f'TR-{atr_n}'].ewm(span=atr_n, adjust=False).mean()

def main():
    # Baixar dados do ativo:
    ticker = input('Ticker: ').upper()
    data_start = input('Inicio (yyyy-mm-dd): ')
    data_end = input('Fim (yyyy-mm-dd): ')
    atr_n = int(input('ATR N: '))

    ativo = yf.download(ticker, start=data_start, end=data_end)

    # Calcula o Retorno
    ativo['Retorno'] = ativo['Close'].pct_change(1)

    # Calcula o ATR
    calculate_atr(ativo, atr_n)

    # Calcula a Média Móvel
    ativo[f'MV-{atr_n}'] = ativo['Close'].rolling(window=atr_n).mean()

    # Exibir os primeiros 20 valores do Ativo sem os NaN
    ativo = ativo.dropna()

    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    print(ativo.head(n=20))

    menu = 12
    while menu != 0:
        print('***** Menu *****')
        print('1: Adquire Dados')
        print('2: Close, Média Móvel, ATR')
        print('3: Histograma')
        print('4: Estatísticas')
        print('5: Imprime as 30 primeiras linhas:')
        print('0: Sair')

        menu = int(input('Qual opção? '))

        if menu == 1:
            ticker = input('Ticker: ').upper()
            data_start = input('Inicio (yyyy-mm-dd): ')
            data_end = input('Fim (yyyy-mm-dd): ')
            atr_n = int(input('ATR N: '))
            ativo = yf.download(ticker, start=data_start, end=data_end)
            ativo['Retorno'] = ativo['Close'].pct_change(1)
            calculate_atr(ativo, atr_n)
            ativo[f'MV-{atr_n}'] = ativo['Close'].rolling(window=atr_n).mean()
            ativo = ativo.dropna()
            print(ativo.head(n=20))

        elif menu == 2 and ativo is not None:
            ativo[f'MV-ATR-{atr_n}'] = ativo['Close'].rolling(window=atr_n).mean()

            plt.figure(figsize=(10, 8))

            plt.subplot(2, 1, 1)
            plt.plot(ativo['Close'], label='Close', color='b')
            plt.plot(ativo[f'MV-ATR-{atr_n}'], label=f'MV-ATR-{atr_n}', color='r')
            plt.ylabel('Close / MV-ATR', color='black')
            plt.tick_params('y', colors='black')
            plt.legend(loc='upper left')

            plt.subplot(2, 1, 2)
            plt.plot(ativo[f'ATR-{atr_n}'], label=f'ATR-{atr_n}', color='g')
            plt.xlabel('Data')
            plt.ylabel('ATR', color='black')
            plt.tick_params('y', colors='black')
            plt.legend(loc='upper right')

            plt.suptitle('Close, Média Móvel, ATR')
            plt.show()

        elif menu == 3 and ativo is not None:
            classes = int(input('Classes: '))

            plt.subplot(221)
            plt.hist(ativo['Retorno Close'].dropna(), bins=classes)
            plt.title('Histograma - Retorno Close')

            plt.subplot(222)
            plt.hist(ativo[f'Retorno ATR-{atr_n}'], bins=classes, color='green')
            plt.title('Histograma - Retorno ATR')

            plt.subplot(223)
            sci.probplot(ativo['Retorno Close'], dist='norm', plot=plt)
            plt.title('QQ-plot - Retorno Close')
            plt.xlabel("")

            plt.subplot(224)
            sci.probplot(ativo[f'Retorno ATR-{atr_n}'], dist='norm', plot=plt)
            plt.title('QQ-plot - Retorno ATR')
            plt.xlabel("")

            plt.tight_layout()
            plt.show()

        elif menu == 4 and ativo is not None:
            ultimo_preco = round(ativo['Close'].iloc[-1], 2)
            maior_preco = round(ativo['High'].max(), 2)
            menor_preco = round(ativo['Low'].min(), 2)
            preco_medio = round(ativo['Close'].mean(), 2)
            max_retorno = round(ativo['Retorno Close'].max()*100, 2)
            dia_max_retorno = ativo['Retorno Close'].idxmax().date()
            min_retorno = round(ativo['Retorno Close'].min()*100, 2)
            dia_min_retorno = ativo['Retorno Close'].idxmin().date()
            volatilidade_precos = round(np.std(ativo['Close'], ddof=0)*100, 2)
            volatilidade_retornos = round(np.std(ativo['Retorno Close'], ddof=0)*100, 2)

            print(f'########## DADOS {ticker} ##########')
            print('------FECHAMENTOS:')
            print(f'ULTIMO: {ultimo_preco}')
            print(f'MAXIMO: {maior_preco}')
            print(f'MINIMO: {menor_preco}')
            print(f'MEDIA: {preco_medio}')
            print('------RETORNOS:')
            print(f'Máximo: {max_retorno}%')
            print(f'DIA MAX: {dia_max_retorno}')
            print(f'MINIMO: {min_retorno}%')
            print(f'DIA MIN: {dia_min_retorno}')
            print('------VOLATILIDADE (desv/pad/pop.):')
            print(f'FECHAMENTOS: {volatilidade_precos}%')
            print(f'RETORNOS: {volatilidade_retornos}%')

        elif menu == 5 and ativo is not None:
            print(ativo.head(n=30))

        elif menu == 0:
            print('Saindo...')

        else:
            print('Opção inválida')

if __name__ == "__main__":
    main()

